# scrape multiple websites

In [1]:
from __future__ import annotations

from datetime import datetime, date
import html
import json
import time
from pathlib import Path
from urllib.parse import urlparse

import pandas as pd
import requests
from bs4 import BeautifulSoup

# --- Selenium imports ---
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# DATE PARAMETERS
START_DATE = date(2020, 1, 1)
END_DATE   = date(2025, 10, 31)

# OUTPUT FILE
OUTPUT_XLSX = "Output/BT/bt_articles_2020-01-01_to_2025-10-31.xlsx"

In [3]:
# All listing pages you want to scrape
LISTING_URLS = [
    "https://www.businesstimes.com.sg/property/residential",
    "https://www.businesstimes.com.sg/keywords/housing",
    "https://www.businesstimes.com.sg/keywords/hdb",
    "https://www.businesstimes.com.sg/keywords/residential",
    "https://www.businesstimes.com.sg/keywords/singapore-residential-property",
    "https://www.businesstimes.com.sg/keywords/residential-property",
]


In [4]:
# Pause timings for listing crawl
SCROLL_PAUSE_SEC = 2.2          # pause after each scroll
NEW_CARDS_SETTLE_SEC = 1.5      # give time for new cards to render
SOFT_SCROLL_CAP = 500           # emergency brake, but main stopping is by logic

# HTTP + scraping
REQUEST_TIMEOUT = 25
PAUSE_BETWEEN_ARTICLE_REQUESTS = 1.0  # politeness delay for article pages
SGT_TZ = "Asia/Singapore"

# checkpoint settings
CHECKPOINT_EVERY = 20  # save to disk after this many new/updated rows

# If True, use Selenium (logged-in) to fetch full article HTML.
USE_SELENIUM_AUTH_FOR_ARTICLES = True

# How many consecutive batches with no *new* links before we give up on a listing
MAX_EMPTY_BATCHES = 10

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )
}

# Accept these meta slots as "published_time"
PUBLISHED_META_SLOTS = [
    ("property", "article:published_time"),
    ("name",     "article:published_time"),
    ("name",     "parsely-pub-date"),
    ("itemprop", "datePublished"),
]

# Some phrases/containers we typically want to skip if they appear in-paragraph
STOP_PHRASES = {
    "Get the BT app", "Get WhatsApp alerts", "Also read:", "Related:", "More on this topic",
    "Have a news tip?", "Sign up", "Subscribe to", "Unlimited access", "Already a subscriber"
}

# global selenium driver
driver: webdriver.Chrome | None = None


# =========================
# Utility helpers
# =========================

def same_domain(u: str) -> bool:
    p = urlparse(u)
    return p.netloc in {"www.businesstimes.com.sg", "businesstimes.com.sg"}


def find_published_meta(soup: BeautifulSoup):
    """
    Return (content, source_meta_selector) for the first usable published_time meta.
    """
    for attr, val in PUBLISHED_META_SLOTS:
        tag = soup.find("meta", attrs={attr: val})
        if tag and tag.get("content"):
            return tag["content"].strip(), f'meta[{attr}="{val}"]'
    return None, None


def parse_iso_like(iso_str: str | None) -> datetime | None:
    if not iso_str:
        return None
    s = iso_str.replace("Z", "+00:00")
    dt = None
    try:
        dt = datetime.fromisoformat(s)
    except Exception:
        for fmt in (
            "%Y-%m-%dT%H:%M:%S%z",
            "%Y-%m-%dT%H:%M:%S.%f%z",
            "%Y-%m-%d %H:%M:%S%z",
            "%Y-%m-%dT%H:%M",
            "%Y-%m-%d %H:%M:%S",
        ):
            try:
                dt = datetime.strptime(s, fmt)
                break
            except Exception:
                continue
    if dt and dt.tzinfo is None:
        from zoneinfo import ZoneInfo
        dt = dt.replace(tzinfo=ZoneInfo(SGT_TZ))
    return dt


def to_sgt(dt: datetime) -> datetime:
    from zoneinfo import ZoneInfo
    return dt.astimezone(ZoneInfo(SGT_TZ))


def within_window(dt: datetime) -> bool:
    return START_DATE <= dt.date() <= END_DATE


def extract_title(soup: BeautifulSoup) -> str:
    h1 = soup.find("h1")
    return h1.get_text(strip=True) if h1 else ""


def normalize_text(text: str) -> str:
    """
    Ensure text is clean UTF-8-compatible and normalise some quotes.
    """
    if not text:
        return ""
    # Make sure it round-trips as UTF-8 (replace any bad bytes with �)
    text = text.encode("utf-8", errors="replace").decode("utf-8")
    # Normalise smart quotes -> straight quotes
    text = (
        text.replace("\u2018", "'")
            .replace("\u2019", "'")
            .replace("\u201c", '"')
            .replace("\u201d", '"')
    )
    return text.strip()


# =========================
# Checkpoint helpers
# =========================

def load_checkpoint() -> pd.DataFrame:
    """
    Load existing checkpoint (if any) so we can resume.
    """
    if Path(OUTPUT_XLSX).exists():
        df = pd.read_excel(OUTPUT_XLSX)
        print(f"[checkpoint] Loaded {len(df)} rows from existing {OUTPUT_XLSX}")
    else:
        df = pd.DataFrame(
            columns=["title", "url", "published_date", "source_url", "content", "content_chars"]
        )
        print("[checkpoint] No existing checkpoint, starting fresh.")
    # Ensure required cols exist
    for col in ["title", "url", "published_date", "source_url", "content", "content_chars"]:
        if col not in df.columns:
            df[col] = "" if col != "content_chars" else 0
    df["content"] = df["content"].fillna("")
    df["content_chars"] = df["content_chars"].fillna(0).astype(int)
    return df


def save_checkpoint(df: pd.DataFrame, label: str = "") -> None:
    """
    Save to a single Excel file as a restartable checkpoint.
    """
    df_copy = df.copy()

    # try to sort by published_date if it parses
    if "published_date" in df_copy.columns:
        with pd.option_context("mode.chained_assignment", None):
            df_copy["_pdt"] = pd.to_datetime(df_copy["published_date"], errors="coerce")
        df_copy = df_copy.sort_values("_pdt").drop(columns=["_pdt"])

    Path(OUTPUT_XLSX).parent.mkdir(parents=True, exist_ok=True)
    df_copy.to_excel(OUTPUT_XLSX, index=False)
    msg_label = f" ({label})" if label else ""
    print(f"[checkpoint] Saved {len(df_copy)} rows to {OUTPUT_XLSX}{msg_label}")


# =========================
# HTTP / HTML fetch helpers
# =========================

def fetch_article_soup_requests(url: str) -> BeautifulSoup | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        # best-effort encoding detection; prefer UTF-8
        if not r.encoding:
            r.encoding = r.apparent_encoding or "utf-8"
        else:
            if r.encoding.lower() in {"latin-1", "iso-8859-1"}:
                r.encoding = "utf-8"
        r.raise_for_status()
        return BeautifulSoup(r.text, "html.parser")
    except Exception as e:
        print(f"  [requests] fetch failed for {url}: {e}")
        return None


def fetch_article_html_requests(url: str) -> str | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        if not r.encoding:
            r.encoding = r.apparent_encoding or "utf-8"
        else:
            if r.encoding.lower() in {"latin-1", "iso-8859-1"}:
                r.encoding = "utf-8"
        r.raise_for_status()
        return r.text
    except Exception as e:
        print(f"  [requests] fetch failed for {url}: {e}")
        return None


def init_selenium_driver() -> webdriver.Chrome:
    """
    Initialise a Selenium Chrome driver with sensible timeouts.
    """
    global driver
    chrome_options = Options()
    # If you want headless for everything, uncomment the next line:
    # chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1280,2000")
    chrome_options.add_argument("--no-sandbox")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    # Avoid hanging forever on page load
    driver.set_page_load_timeout(60)
    return driver


def ensure_login_for_articles():
    """
    If using Selenium for article content, open BT home page once for login.
    """
    global driver
    if driver is None:
        init_selenium_driver()

    try:
        driver.get("https://www.businesstimes.com.sg/")
    except TimeoutException:
        print("[login] Page load timeout while opening BT home page for login.")
    except WebDriverException as e:
        print(f"[login] WebDriver error while opening BT home page: {e}")

    print(
        "\nA Chrome window has opened. Please log in to The Business Times with your subscription.\n"
        "Once you can access a paywalled article in that window, return to the terminal and press Enter."
    )
    input("Press Enter after logging in successfully... ")


def fetch_html_selenium(url: str) -> str | None:
    global driver
    if driver is None:
        raise RuntimeError("Selenium driver not initialised")
    try:
        driver.get(url)
        time.sleep(2.0)
        return driver.page_source
    except TimeoutException:
        print(f"  [selenium] Page load timeout for {url}, skipping.")
        return None
    except WebDriverException as e:
        print(f"  [selenium] WebDriver error for {url}: {e}")
        return None


# =========================
# Content extraction
# =========================

def parse_jsonld_article_body(soup: BeautifulSoup) -> str | None:
    """
    Try to get articleBody from JSON-LD if present.
    """
    try:
        for tag in soup.find_all("script", type="application/ld+json"):
            txt = tag.string or tag.get_text()
            if not txt:
                continue
            data = json.loads(txt.strip())
            candidates = data if isinstance(data, list) else [data]
            for obj in candidates:
                if not isinstance(obj, dict):
                    continue
                t = (obj.get("@type") or "").lower()
                if "article" in t or "newsarticle" in t or "reportage" in t:
                    body = obj.get("articleBody")
                    if body:
                        return html.unescape(body).strip()
    except Exception:
        pass
    return None


def extract_article_text_from_dom(soup: BeautifulSoup) -> str:
    """
    Fallback: collect paragraphs from <article> or main content container.
    """
    root = soup.find("article")
    if root is None:
        for sel in ["main", "section", "div[role='main']", "div.content", "div.article"]:
            root = soup.select_one(sel)
            if root:
                break
    if root is None:
        root = soup

    paras: list[str] = []
    for p in root.find_all("p"):
        txt = p.get_text(" ", strip=True)
        if not txt:
            continue
        if any(stop in txt for stop in STOP_PHRASES):
            continue
        paras.append(txt)

    if not paras:
        for p in soup.find_all("p"):
            txt = p.get_text(" ", strip=True)
            if txt and not any(stop in txt for stop in STOP_PHRASES):
                paras.append(txt)

    content = "\n\n".join(paras).strip()
    content = html.unescape(content)
    return content


def extract_article_content(html_text: str) -> str:
    soup = BeautifulSoup(html_text, "html.parser")

    body = parse_jsonld_article_body(soup)
    if body and len(body) > 80:  # sanity check
        return normalize_text(body)

    return normalize_text(extract_article_text_from_dom(soup))


# =========================
# Listing crawl (find links + dates)
# =========================

def crawl_listings(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crawl all LISTING_URLS with Selenium infinite scroll, collect article links
    and their published_date, and append to df (avoiding duplicates).
    """
    global driver
    if driver is None:
        init_selenium_driver()

    seen_links = set(
        url for url in df.get("url", pd.Series([])).dropna().astype(str).tolist()
    )
    print(f"[listing] Starting with {len(seen_links)} URLs already in checkpoint.")

    new_since_ckpt = 0

    for LISTING_URL in LISTING_URLS:
        print("\n\n==============================")
        print(f"Scraping listing: {LISTING_URL}")
        print("==============================")

        # Load listing page with error handling
        try:
            driver.get(LISTING_URL)
        except TimeoutException:
            print(f"[listing] Page load timeout for {LISTING_URL}, skipping this listing.")
            continue
        except WebDriverException as e:
            print(f"[listing] WebDriver error while loading {LISTING_URL}: {e}")
            continue

        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href]"))
            )
        except TimeoutException:
            print(f"[listing] Timed out waiting for anchors on {LISTING_URL}, skipping.")
            continue
        except Exception as e:
            print(f"[listing] Failed to find anchors on {LISTING_URL}: {e}")
            continue

        oldest_seen_this_listing = None
        batches_since_last_new_in_window = 0
        batch = 0

        while True:
            batch += 1
            # emergency brake, just in case
            if batch > SOFT_SCROLL_CAP:
                print(f"\nHit soft scroll cap ({SOFT_SCROLL_CAP}) on {LISTING_URL}. Stopping.")
                break

            # Scroll and let new cards load
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_SEC)
            time.sleep(NEW_CARDS_SETTLE_SEC)

            # Debug: see if page height is still increasing
            try:
                current_height = driver.execute_script("return document.body.scrollHeight;")
                print(f"[debug] Listing {LISTING_URL} | Batch {batch}: scrollHeight = {current_height}")
            except WebDriverException:
                pass

            try:
                anchors = driver.find_elements(By.CSS_SELECTOR, "a[href]")
            except WebDriverException as e:
                print(f"[listing] Error while reading anchors on {LISTING_URL} (batch {batch}): {e}")
                break

            hrefs = set()
            for a in anchors:
                try:
                    href = a.get_attribute("href")
                except Exception:
                    continue
                if not href:
                    continue
                if not same_domain(href):
                    continue
                hrefs.add(href)

            new_links = sorted([u for u in hrefs if u not in seen_links])
            kept_this_batch = 0
            checked_meta_this_batch = 0
            drop_before_this_batch = 0
            drop_after_this_batch = 0
            no_meta_this_batch = 0
            errors_this_batch = 0
            oldest_this_batch = None

            for u in new_links:
                seen_links.add(u)

                path = urlparse(u).path.rstrip("/")
                if path in {"", "/", "/property", "/property/residential", "/property/commercial-industrial"}:
                    continue

                art_soup = fetch_article_soup_requests(u)
                if art_soup is None:
                    errors_this_batch += 1
                    continue

                iso, src = find_published_meta(art_soup)
                if not iso:
                    no_meta_this_batch += 1
                    continue

                checked_meta_this_batch += 1
                dt = parse_iso_like(iso)
                if not dt:
                    continue

                sgt_dt = to_sgt(dt)
                if (oldest_this_batch is None) or (sgt_dt.date() < oldest_this_batch):
                    oldest_this_batch = sgt_dt.date()
                if (oldest_seen_this_listing is None) or (sgt_dt.date() < oldest_seen_this_listing):
                    oldest_seen_this_listing = sgt_dt.date()

                if within_window(sgt_dt):
                    title = extract_title(art_soup)
                    published_str = sgt_dt.strftime("%Y-%m-%d %H:%M")
                    df = pd.concat(
                        [
                            df,
                            pd.DataFrame(
                                [
                                    {
                                        "title": title,
                                        "url": u,
                                        "published_date": published_str,
                                        "source_url": LISTING_URL,
                                        "content": "",
                                        "content_chars": 0,
                                    }
                                ]
                            ),
                        ],
                        ignore_index=True,
                    )
                    kept_this_batch += 1
                    new_since_ckpt += 1
                elif sgt_dt.date() < START_DATE:
                    drop_before_this_batch += 1
                else:
                    drop_after_this_batch += 1

                time.sleep(PAUSE_BETWEEN_ARTICLE_REQUESTS)

            print(f"\n--- Listing {LISTING_URL} | Batch {batch} summary ---")
            print(f"New links discovered:      {len(new_links)}")
            print(f"Meta-backed links checked: {checked_meta_this_batch}")
            print(f"Kept (in window):          {kept_this_batch}")
            print(f"Dropped before window:     {drop_before_this_batch}")
            print(f"Dropped after window:      {drop_after_this_batch}")
            print(f"No published_time meta:    {no_meta_this_batch}")
            print(f"Errors fetching articles:  {errors_this_batch}")
            print(f"Oldest date this batch:    {oldest_this_batch or '(none)'}")
            print(f"Oldest seen this listing:  {oldest_seen_this_listing or '(none)'}")

            # New empty-batch logic: only care whether there were ANY new_links
            if new_links:
                batches_since_last_new_in_window = 0
            else:
                batches_since_last_new_in_window += 1

            if batches_since_last_new_in_window >= MAX_EMPTY_BATCHES:
                print(
                    f"\nNo new links for {MAX_EMPTY_BATCHES} consecutive batches on this listing. "
                    f"Stopping at batch {batch}."
                )
                break

            if new_since_ckpt >= CHECKPOINT_EVERY:
                save_checkpoint(df, label="after listing batch")
                new_since_ckpt = 0

    save_checkpoint(df, label="after all listings")
    return df


# =========================
# Article content scrape
# =========================

def fill_article_contents(df: pd.DataFrame) -> pd.DataFrame:
    """
    For rows with empty content/content_chars == 0, fetch and fill article body.
    Uses Selenium (logged-in) or requests depending on config.
    """
    global driver

    if USE_SELENIUM_AUTH_FOR_ARTICLES:
        ensure_login_for_articles()

    to_fill = df.index[df["content_chars"].fillna(0) == 0].tolist()
    print(f"[articles] Need to fetch content for {len(to_fill)} articles.")

    updated_since_ckpt = 0
    total = len(to_fill)

    for idx_pos, idx in enumerate(to_fill, start=1):
        url = str(df.at[idx, "url"])
        if not url:
            continue

        print(f"[articles] ({idx_pos}/{total}) Fetching: {urlparse(url).path}")
        time.sleep(PAUSE_BETWEEN_ARTICLE_REQUESTS)

        if USE_SELENIUM_AUTH_FOR_ARTICLES:
            html_text = fetch_html_selenium(url)
        else:
            html_text = fetch_article_html_requests(url)

        if not html_text:
            print("    -> fetch failed; leaving content empty")
            df.at[idx, "content"] = df.at[idx, "content"] or ""
            df.at[idx, "content_chars"] = len(df.at[idx, "content"])
            continue

        content = extract_article_content(html_text)
        n_chars = len(content)
        print(f"    -> {n_chars} characters")

        df.at[idx, "content"] = content
        df.at[idx, "content_chars"] = n_chars

        updated_since_ckpt += 1
        if updated_since_ckpt >= CHECKPOINT_EVERY:
            save_checkpoint(df, label="during article content scrape")
            updated_since_ckpt = 0

    save_checkpoint(df, label="final with content")
    return df


# =========================
# main
# =========================

def main():
    global driver
    df = load_checkpoint()

    # 1) crawl listings (adds any missing links + published_date)
    df = crawl_listings(df)

    # 2) scrape article contents for rows with empty content
    df = fill_article_contents(df)

    if driver is not None:
        driver.quit()
        driver = None

    print("\nDone.")
    print(f"Total rows: {len(df)}")
    print(f"Single output file (with contents): {OUTPUT_XLSX}")


if __name__ == "__main__":
    main()


[checkpoint] Loaded 53 rows from existing Output/BT/bt_articles_2020-01-01_to_2025-10-31.xlsx
[listing] Starting with 53 URLs already in checkpoint.


Scraping listing: https://www.businesstimes.com.sg/property/residential
[debug] Listing https://www.businesstimes.com.sg/property/residential | Batch 1: scrollHeight = 0

--- Listing https://www.businesstimes.com.sg/property/residential | Batch 1 summary ---
New links discovered:      0
Meta-backed links checked: 0
Kept (in window):          0
Dropped before window:     0
Dropped after window:      0
No published_time meta:    0
Errors fetching articles:  0
Oldest date this batch:    (none)
Oldest seen this listing:  (none)
[debug] Listing https://www.businesstimes.com.sg/property/residential | Batch 2: scrollHeight = 3411

--- Listing https://www.businesstimes.com.sg/property/residential | Batch 2 summary ---
New links discovered:      124
Meta-backed links checked: 15
Kept (in window):          0
Dropped before window:     0
Dropped af

Press Enter after logging in successfully...  


[articles] Need to fetch content for 744 articles.
[articles] (1/744) Fetching: /opinion-features/trade-tariffs-may-remove-fomo-among-new-condo-buyers-open-way-relax-absd
    -> 6200 characters
[articles] (2/744) Fetching: /opinion-features/will-deglobalisation-and-business-uncertainty-hurt-buying-power-good-class-bungalow-properties
    -> 6355 characters
[articles] (3/744) Fetching: /property/cdl-places-s1132-psf-ppr-top-bid-plot-next-lakeside-mrt-station
    -> 4928 characters
[articles] (4/744) Fetching: /property/elias-greens-collective-sale-closes-no-bids
    -> 1642 characters
[articles] (5/744) Fetching: /property/hillcrest-arcadias-s920-million-en-bloc-tender-closes-no-bids
    -> 2664 characters
[articles] (6/744) Fetching: /property/lornie-road-bungalow-late-property-developer-market-s82-million-guide-price
    -> 2776 characters
[articles] (7/744) Fetching: /property/mortgagee-listings-climb-q1-interest-rate-strains-hit-market-knight-frank
    -> 3218 characters
[articles] 

# scrape list of BT article links from each page

In [11]:
# ================= identified list of websites =================
# https://www.businesstimes.com.sg/keywords/hdb
# https://www.businesstimes.com.sg/keywords/housing
# https://www.businesstimes.com.sg/keywords/residential
# https://www.businesstimes.com.sg/keywords/residential-property
# https://www.businesstimes.com.sg/keywords/singapore-residential-property
# https://www.businesstimes.com.sg/property/residential

LISTING_URL = "https://www.businesstimes.com.sg/property/residential"
OUT_CSV     = "Output/BT/links_property_residential.csv"

In [12]:
from datetime import datetime, date

START_DATE = date(2020, 1, 1)
END_DATE   = date(2025, 10, 31)

import csv
import time
import os             
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup

# --- Selenium imports ---
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# How many scroll-batches at most (safety cap)
MAX_SCROLL_BATCHES = 5000

# Pause timings
SCROLL_PAUSE_SEC = 1.2          # pause after each scroll
NEW_CARDS_SETTLE_SEC = 1.5      # give time for new cards to render
REQUEST_TIMEOUT = 20
PAUSE_BETWEEN_ARTICLE_REQUESTS = 0.3

# Accept these meta slots as "published_time"
PUBLISHED_META_SLOTS = [
    ("property", "article:published_time"),
    ("name",     "article:published_time"),
    ("name",     "parsely-pub-date"),
    ("itemprop", "datePublished"),
]

SGT_TZ = "Asia/Singapore"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )
}

# ============== Checkpoint helpers (NEW) ==============

def load_existing_rows_and_seen(out_csv):
    """
    If OUT_CSV already exists, load the rows and build a seen_links set
    so we don't re-scrape the same URLs on the next run.
    """
    rows = []
    seen = set()
    if os.path.exists(out_csv):
        with open(out_csv, newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for r in reader:
                # Expecting columns: title, url, published_sgt
                rows.append(r)
                if "url" in r and r["url"]:
                    seen.add(r["url"])
        print(f"[resume] Loaded {len(rows)} existing rows from {out_csv}.")
    else:
        print("[fresh] No existing CSV found, starting new run.")
    return rows, seen

def save_checkpoint(rows, out_csv):
    """
    Save all current rows to OUT_CSV (sorted by published_sgt).
    Called after each batch so we can resume if the script stops.
    """
    rows_sorted = sorted(rows, key=lambda r: r["published_sgt"])
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["title", "url", "published_sgt"])
        w.writeheader()
        w.writerows(rows_sorted)
    print(f"[checkpoint] Saved {len(rows_sorted)} rows to {out_csv}")

# ================= Helpers =================
def to_abs(base, href):
    if not href:
        return None
    return href if href.startswith(("http://", "https://")) else urljoin(base, href)

def same_domain(u):
    p = urlparse(u)
    return p.netloc in {"www.businesstimes.com.sg", "businesstimes.com.sg"}

def find_published_meta(soup):
    for attr, val in PUBLISHED_META_SLOTS:
        tag = soup.find("meta", attrs={attr: val})
        if tag and tag.get("content"):
            return tag["content"].strip(), f'meta[{attr}="{val}"]'
    return None, None

def parse_iso_like(iso_str):
    if not iso_str:
        return None
    s = iso_str.replace("Z", "+00:00")
    dt = None
    try:
        dt = datetime.fromisoformat(s)
    except Exception:
        for fmt in (
            "%Y-%m-%dT%H:%M:%S%z",
            "%Y-%m-%dT%H:%M:%S.%f%z",
            "%Y-%m-%d %H:%M:%S%z",
            "%Y-%m-%dT%H:%M",
            "%Y-%m-%d %H:%M:%S",
        ):
            try:
                dt = datetime.strptime(s, fmt)
                break
            except Exception:
                continue
    if dt and dt.tzinfo is None:
        from zoneinfo import ZoneInfo
        dt = dt.replace(tzinfo=ZoneInfo(SGT_TZ))
    return dt

def to_sgt(dt):
    from zoneinfo import ZoneInfo
    return dt.astimezone(ZoneInfo(SGT_TZ))

def within_window(dt):
    return START_DATE <= dt.date() <= END_DATE

def fetch_article_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def extract_title(soup):
    h1 = soup.find("h1")
    return h1.get_text(strip=True) if h1 else ""

# ========= Load checkpoint BEFORE Selenium starts =========
rows, seen_links = load_existing_rows_and_seen(OUT_CSV)   # <<< NEW >>>

# For early-stopping heuristics
global_oldest_seen = None
batches_since_last_new_in_window = 0

# ================= Selenium setup (fixed) =================
chrome_options = Options()
chrome_options.add_argument("--headless=new")  # comment this if you want to see the browser
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1280,2000")
chrome_options.add_argument("--no-sandbox")

from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(LISTING_URL)

# Wait for first batch of cards to appear
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "a[href]"))
    )
except Exception:
    driver.quit()
    raise SystemExit("Failed to load the listing page or find anchors.")

for batch in range(1, MAX_SCROLL_BATCHES + 1):
    # Scroll to bottom to trigger lazy load/infinite scroll
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_SEC)
    time.sleep(NEW_CARDS_SETTLE_SEC)

    # Collect anchors currently in DOM
    anchors = driver.find_elements(By.CSS_SELECTOR, "a[href]")
    hrefs = set()
    for a in anchors:
        try:
            href = a.get_attribute("href")
        except Exception:
            continue
        if not href:
            continue
        if not same_domain(href):
            continue
        hrefs.add(href)

    # Process only NEW links (dedup across runs via seen_links)
    new_links = sorted([u for u in hrefs if u not in seen_links])
    if not new_links and batches_since_last_new_in_window >= 2:
        print(f"\nNo new links for 2 batches. Stopping at batch {batch}.")
        break

    kept_this_batch = 0
    checked_meta_this_batch = 0
    drop_before_this_batch = 0
    drop_after_this_batch = 0
    no_meta_this_batch = 0
    errors_this_batch = 0
    oldest_this_batch = None

    for u in new_links:
        seen_links.add(u)

        # Quick cheap skip for obvious section roots
        path = urlparse(u).path.rstrip("/")
        if path in {"", "/", "/property", "/property/residential", "/property/commercial-industrial"}:
            continue

        # Fetch article page and check for published_time meta
        try:
            art = fetch_article_soup(u)
        except Exception:
            errors_this_batch += 1
            continue

        iso, src = find_published_meta(art)
        if not iso:
            no_meta_this_batch += 1
            continue

        checked_meta_this_batch += 1
        dt = parse_iso_like(iso)
        if not dt:
            # meta exists but unparsable; treat as no-date
            continue

        sgt_dt = to_sgt(dt)
        if (oldest_this_batch is None) or (sgt_dt.date() < oldest_this_batch):
            oldest_this_batch = sgt_dt.date()
        if (global_oldest_seen is None) or (sgt_dt.date() < global_oldest_seen):
            global_oldest_seen = sgt_dt.date()

        if within_window(sgt_dt):
            title = extract_title(art)
            rows.append({
                "title": title,
                "url": u,
                "published_sgt": sgt_dt.strftime("%Y-%m-%d %H:%M"),
            })
            kept_this_batch += 1
        elif sgt_dt.date() < START_DATE:
            drop_before_this_batch += 1
        else:
            drop_after_this_batch += 1

        time.sleep(PAUSE_BETWEEN_ARTICLE_REQUESTS)

    # ---- Batch summary ----
    print(f"\n--- Batch {batch} summary ---")
    print(f"New links discovered:      {len(new_links)}")
    print(f"Meta-backed links checked: {checked_meta_this_batch}")
    print(f"Kept (in window):          {kept_this_batch}")
    print(f"Dropped before window:     {drop_before_this_batch}")
    print(f"Dropped after window:      {drop_after_this_batch}")
    print(f"No published_time meta:    {no_meta_this_batch}")
    print(f"Errors fetching articles:  {errors_this_batch}")
    print(f"Oldest date this batch:    {oldest_this_batch or '(none)'}")
    print(f"Global oldest seen:        {global_oldest_seen or '(none)'}")

    # early-stop rule:
    if kept_this_batch > 0:
        batches_since_last_new_in_window = 0
    else:
        batches_since_last_new_in_window += 1

    # <<< NEW: save checkpoint after every batch >>>
    save_checkpoint(rows, OUT_CSV)

    if (global_oldest_seen is not None) and (global_oldest_seen < START_DATE) and (batches_since_last_new_in_window >= 2):
        print("\nReached older-than-start dates and no new in-window items for 2 batches. Stopping.")
        break

# Tidy up Selenium
driver.quit()

# ===== Final summary (CSV already saved by last checkpoint) =====
print("\n=== Overall ===")
print(f"Total kept (in window): {len(rows)}")
print(f"Final CSV: {OUT_CSV}")


[fresh] No existing CSV found, starting new run.

--- Batch 1 summary ---
New links discovered:      140
Meta-backed links checked: 30
Kept (in window):          12
Dropped before window:     0
Dropped after window:      18
No published_time meta:    101
Errors fetching articles:  0
Oldest date this batch:    2025-09-04
Global oldest seen:        2025-09-04
[checkpoint] Saved 12 rows to Output/BT/links_property_residential.csv

--- Batch 2 summary ---
New links discovered:      10
Meta-backed links checked: 10
Kept (in window):          10
Dropped before window:     0
Dropped after window:      0
No published_time meta:    0
Errors fetching articles:  0
Oldest date this batch:    2025-07-11
Global oldest seen:        2025-07-11
[checkpoint] Saved 22 rows to Output/BT/links_property_residential.csv

--- Batch 3 summary ---
New links discovered:      10
Meta-backed links checked: 10
Kept (in window):          10
Dropped before window:     0
Dropped after window:      0
No published_time 

# combine all the links and remove duplicates

In [13]:
import pandas as pd

In [17]:
link1 = pd.read_csv('Output/BT/links_hdb.csv')
link2 = pd.read_csv('Output/BT/links_housing.csv')
link3 = pd.read_csv('Output/BT/links_property_residential.csv')
link4 = pd.read_csv('Output/BT/links_residential.csv')
link5 = pd.read_csv('Output/BT/links_residential-property.csv')
link6 = pd.read_csv('Output/BT/links_singapore-residential-property.csv')

In [26]:
all_links = pd.concat([link1, link2, link3, link4, link5, link6])

In [27]:
all_links.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4267 entries, 0 to 584
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          4267 non-null   object
 1   url            4267 non-null   object
 2   published_sgt  4267 non-null   object
dtypes: object(3)
memory usage: 133.3+ KB


In [29]:
# remove duplicates
all_links = all_links.drop_duplicates(subset=["url"], keep="first")
len(all_links)

2925

In [34]:
# rename column for standardisation
all_links.rename(columns={'published_sgt':'published_date'}, inplace=True)

In [39]:
all_links.to_excel('Output/BT/all_links.xlsx')

# scrape what has not been scraped

In [31]:
# originally scraped
df = pd.read_excel('Output/BT/bt_articles_2020-01-01_to_2025-10-31.xlsx')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           797 non-null    object
 1   url             797 non-null    object
 2   published_date  797 non-null    object
 3   source_url      797 non-null    object
 4   content         797 non-null    object
 5   content_chars   797 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 37.5+ KB


In [37]:
# check what links have not been scraped
original_links = df[['title', 'url', 'published_date']]
add_links = all_links[~all_links["url"].isin(original_links["url"])]
len(add_links)


2131

In [40]:
add_links.to_excel('Output/BT/add_links.xlsx')

# scrape the additional links

In [44]:
from __future__ import annotations

import time
import html
import json
from pathlib import Path
from urllib.parse import urlparse

import pandas as pd
import requests
from bs4 import BeautifulSoup

# --- Selenium imports ---
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# ============================================================
# CONFIG
# ============================================================

INPUT_XLSX = "Output/BT/add_links.xlsx"
OUTPUT_XLSX = "Output/BT/add_links.xlsx"

USE_SELENIUM = True                # set to False if you want requests only
REQUEST_TIMEOUT = 25
PAUSE = 1.0                        # politeness delay between articles (seconds)

CHECKPOINT_EVERY = 20              # ✅ save to Excel after this many updates

STOP_PHRASES = {
    "Get the BT app", "Get WhatsApp alerts", "Also read:", "Related:",
    "More on this topic", "Have a news tip?", "Sign up", "Subscribe to",
    "Unlimited access", "Already a subscriber"
}

# <<< NEW: patterns that indicate a broken / error / non-article page >>>
ERROR_SNIPPETS = [
    "The page you are looking for doesn't exist, or it may have been removed.",
    "The page you are looking for doesn’t exist, or it may have been removed.",  # curly apostrophe
    "Search\n\nM\n\nThe page you are looking for doesn't exist, or it may have been removed.",
]

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )
}

driver = None


# ============================================================
# Setup helpers
# ============================================================

def init_driver():
    global driver
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1280,2000")
    chrome_options.add_argument("--no-sandbox")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(60)
    return driver


def ensure_login():
    """
    Opens Business Times home page so user can manually log in once.
    """
    if driver is None:
        init_driver()

    home = "https://www.businesstimes.com.sg/"
    try:
        driver.get(home)
    except Exception as e:
        print(f"[login] Error opening home page: {e}")

    print("\nA Chrome window has opened.")
    print("If needed, log in to BT so paywalled articles are accessible.")
    input("Press Enter after login is completed… ")


# ============================================================
# Fetch helpers
# ============================================================

def fetch_html_requests(url: str):
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        r.encoding = r.apparent_encoding or "utf-8"
        return r.text
    except Exception as e:
        print(f"[requests] Error for {url}: {e}")
        return None


def fetch_html_selenium(url: str):
    global driver
    try:
        driver.get(url)
        time.sleep(2)
        return driver.page_source
    except Exception as e:
        print(f"[selenium] Error for {url}: {e}")
        return None


# ============================================================
# Content Extraction
# ============================================================

def parse_jsonld(soup: BeautifulSoup):
    """
    Extract articleBody via JSON-LD if present.
    """
    try:
        for tag in soup.find_all("script", type="application/ld+json"):
            txt = tag.string or tag.get_text()
            if not txt:
                continue
            data = json.loads(txt.strip())
            objs = data if isinstance(data, list) else [data]
            for obj in objs:
                if isinstance(obj, dict) and "articleBody" in obj:
                    return html.unescape(obj["articleBody"])
    except Exception:
        pass
    return None


def extract_text(soup: BeautifulSoup):
    root = (
        soup.find("article")
        or soup.select_one("main")
        or soup.select_one("div.content")
        or soup
    )

    paras = []
    for p in root.find_all("p"):
        txt = p.get_text(" ", strip=True)
        if txt and not any(stop in txt for stop in STOP_PHRASES):
            paras.append(txt)

    return "\n\n".join(paras).strip()


def extract_content(html_text: str):
    soup = BeautifulSoup(html_text, "html.parser")

    # Try JSON-LD first
    body = parse_jsonld(soup)
    if body and len(body) > 50:
        return body.strip()

    return extract_text(soup)


# ============================================================
# Main scraping logic with checkpoints
# ============================================================

def main():
    global driver

    # ---------- Load Excel ----------
    df = pd.read_excel(INPUT_XLSX)
    print(f"Loaded {len(df)} rows from {INPUT_XLSX}")

    # Ensure content columns exist and are numeric
    if "content" not in df.columns:
        df["content"] = ""
    if "content_chars" not in df.columns:
        df["content_chars"] = 0

    df["content"] = df["content"].fillna("")
    df["content_chars"] = df["content_chars"].fillna(0).astype(int)

    # ---------- Optionally login for Selenium ----------
    if USE_SELENIUM:
        ensure_login()

    updated_since_ckpt = 0
    total_rows = len(df)

    for idx, row in df.iterrows():
        url = row["url"]
        if not isinstance(url, str) or not url.strip():
            continue

        # --- Decide if this row needs re-scraping ---  <<< NEW >>>
        existing_content = str(row.get("content", "") or "")
        existing_chars = int(row.get("content_chars", 0) or 0)

        is_empty = (existing_chars == 0) or (existing_content.strip() == "")
        has_error_snippet = any(snippet in existing_content for snippet in ERROR_SNIPPETS)

        needs_rescrape = is_empty or has_error_snippet

        # If content looks OK (non-empty and not error page), skip
        if not needs_rescrape:
            continue

        print(f"\n[{idx+1}/{total_rows}] Scraping: {url}")
        if has_error_snippet:
            print(" -> Existing content looks like error page; re-scraping.")

        if USE_SELENIUM:
            html_text = fetch_html_selenium(url)
        else:
            html_text = fetch_html_requests(url)

        if not html_text:
            print(" -> Failed to fetch")
            continue

        content = extract_content(html_text)
        df.at[idx, "content"] = content
        df.at[idx, "content_chars"] = len(content)
        print(f" -> {len(content)} chars extracted")

        updated_since_ckpt += 1

        # ---------- Checkpoint ----------
        if updated_since_ckpt >= CHECKPOINT_EVERY:
            Path(OUTPUT_XLSX).parent.mkdir(parents=True, exist_ok=True)
            df.to_excel(OUTPUT_XLSX, index=False)
            print(f"[checkpoint] Saved after {updated_since_ckpt} updates.")
            updated_since_ckpt = 0

        time.sleep(PAUSE)

    # ---------- Final save ----------
    Path(OUTPUT_XLSX).parent.mkdir(parents=True, exist_ok=True)
    df.to_excel(OUTPUT_XLSX, index=False)
    print(f"\nFinal save to {OUTPUT_XLSX}")

    if driver:
        driver.quit()


if __name__ == "__main__":
    main()


Loaded 2131 rows from Output/BT/add_links.xlsx

A Chrome window has opened.
If needed, log in to BT so paywalled articles are accessible.


Press Enter after login is completed…  



[120/2131] Scraping: https://www.businesstimes.com.sg/companies-markets/banking-finance/hdb-issues-s800m-notes-due-in-2026
 -> Existing content looks like error page; re-scraping.
 -> 938 chars extracted

[153/2131] Scraping: https://www.businesstimes.com.sg/property/in-weighing-hdb-lottery-effect-singapore-mulls-raising-subsidies-imposing-clawbacks
 -> Existing content looks like error page; re-scraping.
 -> 6869 chars extracted

[250/2131] Scraping: https://www.businesstimes.com.sg/property/hdb-resale-prices-climb-at-a-slower-pace-in-q1-2022
 -> Existing content looks like error page; re-scraping.
 -> 2641 chars extracted

[560/2131] Scraping: https://www.businesstimes.com.sg/international/global/residential-assets-grab-spotlight-in-private-equity-real-estate-market-panel
 -> 5342 chars extracted

[668/2131] Scraping: https://www.businesstimes.com.sg/property/chinatown-shophouses-put-market-s110m-killiney-road-site-sale-s76m
 -> Existing content looks like error page; re-scraping.
 

# combine all scraped data

In [14]:
import pandas as pd

df1 = pd.read_excel("Output/BT/bt_articles_2020-01-01_to_2025-10-31.xlsx")
df2 = pd.read_excel("Output/BT/add_links.xlsx")

In [15]:
# standardise the columns
df1 = df1.drop(columns=['source_url']) # no need to categorise source_url in future
df2 = df2.drop(columns=['Unnamed: 0']) # index set to False in future


In [16]:
# concat both dfs
df = pd.concat([df1, df2])

In [17]:
# check if any duplicates
df['title'].duplicated().any()


True

In [18]:
# there are duplicates because of minor website naming differences, e.g., https://www.businesstimes.com.sg/property/singapore-homes-to-see-firm-rents-more-leasing-deals-when-border-with-malaysia-reopens-further and https://www.businesstimes.com.sg/property/singapore-homes-to-see-firm-rents-more-leasing-deals-when-border-with-malaysia-reopens-further1

# convert published_date into a datetime column
df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')

# drop duplicates, keep the most updated article
df = df.sort_values('published_date', ascending=False)
df = df.drop_duplicates(subset='title', keep='first')


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2894 entries, 796 to 0
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           2894 non-null   object        
 1   url             2894 non-null   object        
 2   published_date  2894 non-null   datetime64[ns]
 3   content         2894 non-null   object        
 4   content_chars   2894 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 135.7+ KB


In [20]:
df.head()

,title,url,published_date,content,content_chars
796,"US mortgage rates drop to 6.17%, lowest since early October 2024",https://www.businesstimes.com.sg/property/us-mortgage-rates-drop-6-17-lowest-early-october-2024,2025-10-31 07:04:00,"This follows Fed's widely anticipated quarter-point reduction; home sales are flat despite growing inventories\n\nShare\n\n[BOSTON] Mortgage rates in the US fell for a fourth week, inching closer to 6 per cent even as economic uncertainty is keeping buyers pinned to the sidelines.\n\nThe average for 30-year fixed loans was 6.17 per cent, down from 6.19 per cent last week and still the lowest level since early October 2024, data from Freddie Mac showed.\n\nRates fell following the US Federal Reserve's widely anticipated quarter-point reduction on Wednesday (Oct 29). Chairman Jerome Powell also cautioned investors not to expect more cuts this year.\n\nThe drop in borrowing costs, along with a surging stock market, would typically predict a surge in demand, and economists had expected to see a slight increase in pending sales of existing homes last month.\n\nThat did not happen: Sales were flat, despite growing inventories, the market blunted by anxieties about tariffs, job security and the ongoing government shutdown.\n\nRates are not likely to move much lower ""absent surprisingly slower economic activity"", Realtor.com chief economist Danielle Hale said, noting that the market has already priced in the most recent cut.\n\nSEE ALSO\n\nEconomic concerns ""could also mean rising rates through the end of the year"", said Bright MLS economist Lisa Sturtevant. ""For prospective buyers who are financially ready, right now could be a sweet spot for lower rates and more inventory"". BLOOMBERG\n\nShare with us your feedback on BT's products and services",1543
795,US pending home sales stall as labour market concerns build,https://www.businesstimes.com.sg/property/us-pending-home-sales-stall-labour-market-concerns-build,2025-10-30 07:39:00,"Mortgage rates have shown signs of stabilising below 6.5%, and home prices are cooling\n\nShare\n\n[ATLANTA] Pending sales of existing US homes stalled in September, suggesting anxiety about the job market kept potential buyers sidelined despite a welcome easing in mortgage rates.\n\nAn index of contract signings held at 74.8 after climbing a revised 4.2 per cent a month earlier to the highest level since March, according to National Association of Realtors (NAR) data released on Wednesday (Oct 29). Economists expected a 1.2 per cent increase, based on the median estimate in a Bloomberg survey.\n\n""A record-high stock market and growing housing wealth in September were not enough to offset a likely softening job market,"" NAR chief economist Lawrence Yun said. Nonetheless, ""mortgage rates are trending towards three-year lows, which should further improve affordability, though the government shutdown could temporarily slow home sales activity"".\n\nEven with middling results for last month, housing economists see the resale market slowly thawing after several years of lacklustre demand. Mortgage rates have shown signs of stabilising below 6.5 per cent, and home prices are cooling.\n\nMoreover, the so-called ""lock-in effect"", in which homeowners resist selling because of their existing low-rate mortgages, is waning and helping to boost inventory.\n\nFigures out on Tuesday from S&P Cotality Case-Shiller showed buyers are gaining leverage in negotiations with sellers. A national measure of home prices rose 1.5 per cent in August from a year ago, the smallest annual gain since mid-2023.\n\nSEE ALSO\n\nBy region, contract signings on previously owned homes rose 1.1 per cent in the South to the highest level since March. Pending sales also climbed in the Northeast, while falling in the West and Midwest.\n\nThe broader housing market has experienced more momentum recently, with new-home sales surging in August to the highest level since 2022. While some economists chalked u

In [22]:
df.to_excel('Output/BT/final_bt_articles_2020-01-01_to_2025-10-31.xlsx', index=False)

# clean scraped data

In [124]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import re

In [125]:
df = pd.read_excel("Output/BT/final_bt_articles_2020-01-01_to_2025-10-31.xlsx")

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2894 entries, 0 to 2893
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           2894 non-null   object        
 1   url             2894 non-null   object        
 2   published_date  2894 non-null   datetime64[ns]
 3   content         2894 non-null   object        
 4   content_chars   2894 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 113.2+ KB


In [127]:
df.head()

,title,url,published_date,content,content_chars
0,"US mortgage rates drop to 6.17%, lowest since early October 2024",https://www.businesstimes.com.sg/property/us-mortgage-rates-drop-6-17-lowest-early-october-2024,2025-10-31 07:04:00,"This follows Fed's widely anticipated quarter-point reduction; home sales are flat despite growing inventories\n\nShare\n\n[BOSTON] Mortgage rates in the US fell for a fourth week, inching closer to 6 per cent even as economic uncertainty is keeping buyers pinned to the sidelines.\n\nThe average for 30-year fixed loans was 6.17 per cent, down from 6.19 per cent last week and still the lowest level since early October 2024, data from Freddie Mac showed.\n\nRates fell following the US Federal Reserve's widely anticipated quarter-point reduction on Wednesday (Oct 29). Chairman Jerome Powell also cautioned investors not to expect more cuts this year.\n\nThe drop in borrowing costs, along with a surging stock market, would typically predict a surge in demand, and economists had expected to see a slight increase in pending sales of existing homes last month.\n\nThat did not happen: Sales were flat, despite growing inventories, the market blunted by anxieties about tariffs, job security and the ongoing government shutdown.\n\nRates are not likely to move much lower ""absent surprisingly slower economic activity"", Realtor.com chief economist Danielle Hale said, noting that the market has already priced in the most recent cut.\n\nSEE ALSO\n\nEconomic concerns ""could also mean rising rates through the end of the year"", said Bright MLS economist Lisa Sturtevant. ""For prospective buyers who are financially ready, right now could be a sweet spot for lower rates and more inventory"". BLOOMBERG\n\nShare with us your feedback on BT's products and services",1543
1,US pending home sales stall as labour market concerns build,https://www.businesstimes.com.sg/property/us-pending-home-sales-stall-labour-market-concerns-build,2025-10-30 07:39:00,"Mortgage rates have shown signs of stabilising below 6.5%, and home prices are cooling\n\nShare\n\n[ATLANTA] Pending sales of existing US homes stalled in September, suggesting anxiety about the job market kept potential buyers sidelined despite a welcome easing in mortgage rates.\n\nAn index of contract signings held at 74.8 after climbing a revised 4.2 per cent a month earlier to the highest level since March, according to National Association of Realtors (NAR) data released on Wednesday (Oct 29). Economists expected a 1.2 per cent increase, based on the median estimate in a Bloomberg survey.\n\n""A record-high stock market and growing housing wealth in September were not enough to offset a likely softening job market,"" NAR chief economist Lawrence Yun said. Nonetheless, ""mortgage rates are trending towards three-year lows, which should further improve affordability, though the government shutdown could temporarily slow home sales activity"".\n\nEven with middling results for last month, housing economists see the resale market slowly thawing after several years of lacklustre demand. Mortgage rates have shown signs of stabilising below 6.5 per cent, and home prices are cooling.\n\nMoreover, the so-called ""lock-in effect"", in which homeowners resist selling because of their existing low-rate mortgages, is waning and helping to boost inventory.\n\nFigures out on Tuesday from S&P Cotality Case-Shiller showed buyers are gaining leverage in negotiations with sellers. A national measure of home prices rose 1.5 per cent in August from a year ago, the smallest annual gain since mid-2023.\n\nSEE ALSO\n\nBy region, contract signings on previously owned homes rose 1.1 per cent in the South to the highest level since March. Pending sales also climbed in the Northeast, while falling in the West and Midwest.\n\nThe broader housing market has experienced more momentum recently, with new-home sales surging in August to the highest level since 2022. While some economists chalked up th

In [128]:
pd.set_option('display.max_colwidth', None)
print(df.loc[df['title'] == "PropertyBT: Selling Singapore’s new dream homes", 'content'].to_string(index=False))

Share\n\nSingaporeans have always had a special interest in real estate investment so GuocoLand's recent success in Lentor might have piqued your interest. It certainly piqued Leslie Yee's interest.\n\nAs host of The Business Times podcast PropertyBT, he sat down with Dora Chng, residential director at GuocoLand to garner insights into their residential developments in the latest podcast episode.\n\nUnearth market trends and insights\n\nThe pair explore why new condo units in Singapore are commanding high prices and selling like hotcakes. With Chng's expert insights, listeners gain a comprehensive understanding of the unique appeal and advantages of owning a brand-new condo compared to resale properties. This episode is packed with data and observations that can help potential buyers and investors make informed decisions.\n\nShe explains the intrinsic value and attraction of new condos, such as modern designs, fresh tenure, and state-of-the-art facilities. She also points out that new 

In [129]:
# remove irrelevant phrases

lines_to_remove = [
    "Share\n\n",
    "SEE ALSO",
    "Copyright SPH Media. All rights reserved.",
    "Share with us your feedback on BT's products and services",
    "\n\nREUTERS\n\nShare with us your feedback on BT's products and services",
    "BLOOMBERG\n\nShare with us your feedback on BT's products and services"
]

pattern = "|".join(re.escape(line) for line in lines_to_remove)

df['content'] = (
    df['content']
    .str.replace(pattern, "", regex=True)
    #.str.replace(r"\s+", " ", regex=True)  # clean extra spaces
    .str.strip()
)

In [130]:
# filter for articles about Singapore

# ensure column is string
df['content'] = df['content'].astype(str)

# phrases to clean
df["content"] = (
    df["content"]
    .str.replace("[but]", "but", case=False)
    .str.replace("[to pay their mortgage]", "to pay their mortgage", case=False)
    .str.replace("[of co-ops]", "of co-ops", case=False)
    .str.replace("[Mercedes]", "Mercedes", case=False)
    .str.replace("[BOE’s]", "BOE's", case=False)
    .str.replace("[on Reddit]", "on Reddit", case=False)
)

# extract location stated within square bracket
df["location"] = df["content"].str.extract(r"\[(.*?)\]")


In [131]:
df["location"].value_counts()

location
SINGAPORE              232
LONDON                  42
BEIJING                 35
WASHINGTON              29
NEW YORK                24
SYDNEY                  17
HONG KONG               14
SEOUL                   11
WELLINGTON              11
BOSTON                  10
ATLANTA                  7
TOKYO                    6
OTTAWA                   5
MADRID                   4
NEW DELHI                4
LOS ANGELES              3
SHANGHAI                 3
FRANKFURT                3
TORONTO                  2
BANGKOK                  2
CHICAGO                  2
PARIS                    2
KUALA LUMPUR             2
HO CHI MINH CITY         2
JAKARTA                  2
BENGALURU                2
MELBOURNE                1
MIAMI                    1
GUATAVITA, COLOMBIA      1
MUMBAI                   1
TAIPEI                   1
COLORADO                 1
DUBLIN                   1
AUCKLAND                 1
GENEVA                   1
BERLIN                   1
LISBON             

In [132]:
# keep only those which are Singapore or NA (not mentioned)
df = df[
    (df["location"] == "SINGAPORE") |
    (df["location"] == "SINGAPORE ") |
    (df["location"].isna()) |
    (df["location"].str.strip() == "")
]


In [133]:
len(df)

2631

In [134]:
# parse 'content' column and check if there are any Singapore-related keywords

# ensure column is string
df['content'] = df['content'].astype(str)

# check if content has Singapore-related keywords
keywords = ["Singapore", "Singapore's", "Singaporeans", "Ministry of National Development", "Housing & Development Board", "Housing and Development Board", "Building and Construction Authority", "Urban Redevelopment Authority",
           "Build-To-Order", "Government Land Sales", "buyer stamp duty", "GuocoLand", "Bukit Sembawang", "Core Central Region", "Rest of Central Region",
           "Outside Central Region", "SRX", "minimum occupation period", "temporary occupation permit", "BTO", "Upper Bukit Timah", "Sixth Avenue", "HDB Flat",
           "CAPITALAND", "HDB flats", "Sunset Way", "Farrer Park", "HDB resale flat", "Bukit Timah", "Good Class Bungalows", "Orchard Boulevard", "town councils",
           "99-year leasehold", "land betterment charge", "Tanah Merah", "Holland Road", "Upper Thomson", "Thomson View", "Namly View", "OrangeTee"]

pattern = r"\b(" + "|".join(re.escape(k) for k in keywords) + r")\b"

df["Singapore_context"] = (
    df["content"]
    .str.contains(pattern, case=False, regex=True)
    .map({True: "yes", False: "no"})
)


In [135]:
df['Singapore_context'].value_counts()

Singapore_context
yes    1395
no     1236
Name: count, dtype: int64

In [137]:
# parse 'title' column and assign the location
# note: code may not capture all the distinct countries, but sufficient for our purpose
# to identify articles of Singapore context

# ----------------------------------------
# 1. Country and demonym keywords
# ----------------------------------------
COUNTRY_KEYWORDS = {
    # East Asia
    "china": "China",
    "chinese": "China",
    "hong kong": "Hong Kong",
    "japan": "Japan",
    "japanese": "Japan",
    "south korea": "South Korea",
    "koreans": "South Korea",
    "korea": "South Korea",

    # South / Southeast Asia
    "india": "India",
    "indian": "India",
    "bangladesh": "Bangladesh",
    "singapore": "Singapore",
    "indonesia": "Indonesia",
    "indonesian": "Indonesia",
    "malaysia": "Malaysia",
    "malaysian": "Malaysia",
    "thailand": "Thailand",
    "thai": "Thailand",
    "vietnam": "Vietnam",
    "vietnamese": "Vietnam",
    "philippines": "Philippines",
    "filipino": "Philippines",

    # Europe
    "united kingdom": "United Kingdom",
    "uk": "United Kingdom",
    "britain": "United Kingdom",
    "british": "United Kingdom",
    "britons": "United Kingdom",
    "great britain": "United Kingdom",
    "england": "United Kingdom",
    "germany": "Germany",
    "german": "Germany",
    "french": "France",
    "france": "France",
    "russian": "Russia",
    "spain": "Spain",
    "europe": "Europe",
    "eu ": "Europe",
    "denmark": "Denmark",
    "sweden": "Sweden",
    "irish": "Ireland",
    "swedes": "Sweden",
    "swedish": "Sweden",
    "finland": "Finland",
    "norway": "Norway",


    # North America
    "united states": "United States",
    "u.s.": "United States",
    "u.s": "United States",
    "u.s.a": "United States",
    "us ": "United States",
    "usa": "United States",
    "america": "United States",
    "american": "United States",
    "canada": "Canada",
    "canadian": "Canada",

    # Middle East (keep broad or map more specifically if you want)
    "saudi arabia": "Saudi Arabia",
    "saudi": "Saudi Arabia",
    "uae": "United Arab Emirates",
    "united arab emirates": "United Arab Emirates",
    "dubai": "United Arab Emirates",
    "abu dhabi": "United Arab Emirates",
    "qatar": "Qatar",
    "doha": "Qatar",
    "turkey": "Türkiye",
    "turkish": "Türkiye",
    "israel": "Israel",
    "pakistan": "Pakistan",

    # Australasia
    "australia": "Australia",
    "australian": "Australia",
    "aussie": "Australia",
    "new zealand": "New Zealand",
    "nz": "New Zealand",
    "kiwi": "New Zealand",
}

# ----------------------------------------
# 2. City → Country dictionary
#    (curated, not everything in the world)
# ----------------------------------------
CITY_TO_COUNTRY = {
    # India
    "mumbai": "India",
    "delhi": "India",
    "new delhi": "India",
    "bengaluru": "India",
    "bangalore": "India",
    "chennai": "India",

    # China
    "shanghai": "China",
    "beijing": "China",
    "shenzhen": "China",
    "guangzhou": "China",
    "chengdu": "China",
    "wuhan": "China",

    # US
    "new york": "United States",
    "new york city": "United States",
    "nyc": "United States",
    "los angeles": "United States",
    "la": "United States",
    "san francisco": "United States",
    "seattle": "United States",
    "chicago": "United States",
    "miami": "United States",
    "brooklyn": "United States",
    "manhattan": "United States",
    "toronto": "Canada",
    "biden": "United States",
    
    # UK / Europe
    "london": "United Kingdom",
    "paris": "France",
    "berlin": "Germany",
    "frankfurt": "Germany",
    "zurich": "Switzerland",
    "rotterdam": "Netherlands",

    # Asia-Pacific
    "tokyo": "Japan",
    "osaka": "Japan",
    "seoul": "South Korea",
    "bangkok": "Thailand",
    "jakarta": "Indonesia",
    "kuala lumpur": "Malaysia",
    "manila": "Philippines",
    "sydney": "Australia",
    "melbourne": "Australia",
    "auckland": "New Zealand",
    "hong kong": "Hong Kong",  # city + territory
}

def _contains_phrase(text_lower: str, phrase: str) -> bool:
    """
    Case-insensitive phrase match with word boundary at the start,
    but not necessarily at the end so that we match 'china’s', 'china's', etc.
    """
    # handle apostrophes: 'china’s' vs 'china's'
    pattern = r"\b" + re.escape(phrase)
    return re.search(pattern, text_lower) is not None


def predict_country_from_title(title: str):
    """
    1. Look for country names / demonyms in the title.
    2. If none, look for known cities and map to countries.
    3. Return a list of unique countries; if none, return ["Unknown"].
    """
    if not isinstance(title, str) or not title.strip():
        return ["Unknown"]

    text = title.strip()
    text_lower = text.lower()

    found_countries = []

    # ----- A. Country / demonym keywords -----
    for key, country in COUNTRY_KEYWORDS.items():
        if _contains_phrase(text_lower, key):
            found_countries.append(country)

    if found_countries:
        # remove duplicates, keep order
        uniq = []
        for c in found_countries:
            if c not in uniq:
                uniq.append(c)
        return uniq

    # ----- B. City → country -----
    for city, country in CITY_TO_COUNTRY.items():
        if _contains_phrase(text_lower, city):
            found_countries.append(country)

    if found_countries:
        uniq = []
        for c in found_countries:
            if c not in uniq:
                uniq.append(c)
        return uniq

    # ----- C. Fallback -----
    return ["Unknown"]


# ----------------------------------------
# 3. Apply to your dataframe and save
# ----------------------------------------
df["predicted_country_from_title"] = df["title"].apply(predict_country_from_title)


In [141]:
# remove those which are not Singapore context
condition = (
    df["location"].isna() &
    (df["Singapore_context"] == "no")
)

df = df[~condition]

In [142]:
len(df)

1395

In [143]:
df.columns

Index(['title', 'url', 'published_date', 'content', 'content_chars',
       'location', 'Singapore_context', 'predicted_country_from_title'],
      dtype='object')

In [144]:
# keep only relevant columns
df = df[['title', 'url', 'published_date', 'content', 'content_chars']]

In [145]:
df.to_excel('Output/BT/cleaned_bt_articles_2020-01-01_to_2025-10-31.xlsx', index=False)